In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import anndata as ad
import dynamo as dyn
import scanpy as sc

# Prepare data

In [12]:
adata = ad.read_h5ad("../08_blastocyst/results/blastocyst_counts.transcript_based.transcript_name.annotated.filtered.h5ad")
adata2 = ad.read_h5ad("../08_blastocyst/results/scanpy_processed_transcript_based.h5ad")
adata = adata[adata.obs.index.isin(adata2.obs.index),:]
adata.obs["CellType"] = adata2.obs["CellType"]
adata.layers["new"] = adata.layers["nascent"]
adata = adata[(adata.obs["s4U"] == 400) & (adata.obs["Time"] == 3) & (adata.obs["ActD"].isna()) & (adata.obs["TC.Ratio"] >= 0)]
sc.pp.filter_cells(adata, min_genes=600)
adata.write("results/dynamo_data_transcript_based.h5ad", compression="gzip")
print(adata)

/home/chenzonggui/tmp/ipykernel_17708/1666605633.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["CellType"] = adata2.obs["CellType"]
/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:163: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_genes"] = number


AnnData object with n_obs × n_vars = 2126 × 73649
    obs: 'Run', 'Barcode', 'Species', 's4U', 'Time', 'ActD', 'Group', 'CellLine', 'Stage', 'Platform', 'Label', 'UMI', 'Cells', 'LibStruct', 'Remark', 'Cell.Reads', 'Trimmed.Reads', 'Trimmed.Ratio', 'Mapped.Reads', 'Mapped.Ratio', 'Mito.Ratio', 'Filtered.Reads', 'Filtered.Ratio', 'FilteredClip.Reads', 'FilteredClip.Ratio', 'UMIs', 'UMIs.2Reads', 'Duplicate.Reads', 'Duplicate.Ratio', 'Unique.Reads', 'Genes', 'Isoforms.Assembled', 'Isoforms.Known', 'AC.Ratio', 'AG.Ratio', 'AT.Ratio', 'CA.Ratio', 'CG.Ratio', 'CT.Ratio', 'GA.Ratio', 'GC.Ratio', 'GT.Ratio', 'TA.Ratio', 'TC.Ratio', 'TG.Ratio', 'Pe', 'Pc', 'PcPe.Ratio', 'UMIs.2Reads.Nascent.2TCs', 'UMIs.2Reads.Nascent.2TCs.Ratio', 'Genes.2Reads', 'Genes.2Reads.Nascent.2TCs', 'CellType', 'n_genes'
    var: 'TranscriptID', 'TranscriptType', 'GeneID', 'GeneName', 'GeneType', 'Chrom', 'Start', 'End', 'Strand'
    layers: 'nascent', 'total', 'new'


# Run dynamo

In [13]:
adata = ad.read_h5ad("results/dynamo_data_transcript_based.h5ad")
dyn.pp.recipe_monocle(adata, tkey="Time")
dyn.tl.dynamics(adata, model='stochastic', cores=3, NTR_vel=True)
dyn.tl.reduceDimension(adata)
dyn.tl.leiden(adata)
dyn.tl.hdbscan(adata)
dyn.tl.louvain(adata)
if True:
    lineage_dict = {"TE-1": ["muralTE", "polarTE"], "ICM-1": ["EPI", "PE"]}
    dyn.tl.gene_wise_confidence(adata, group='CellType', lineage_dict=lineage_dict, ekey="M_t", vkey="velocity_T")
    adata = adata[:,(adata.var["avg_prog_confidence"] > 0) & (adata.var["avg_mature_confidence"] > 0.5)]
dyn.tl.cell_velocities(adata, calc_rnd_vel=True)
dyn.vf.VectorField(adata, basis='umap')

|-----? dynamo.preprocessing.deprecated is deprecated.
|-----> recipe_monocle_keep_filtered_cells_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_cells_key=True
|-----> recipe_monocle_keep_filtered_genes_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_genes_key=True
|-----> recipe_monocle_keep_raw_layers_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_raw_layers_key=True
|-----> apply Monocole recipe to adata...


/home/chenzonggui/tmp/ipykernel_17708/2130489997.py:2: DeprecationWarning: recipe_monocle is deprecated and will be removed in a future release. Please update your code to use the new replacement function.
  dyn.pp.recipe_monocle(adata, tkey="Time")


|-----> ensure all cell and variable names unique.
|-----> ensure all data in different layers in csr sparse matrix format.
|-----> ensure all labeling data properly collapased
|-----> detected experiment type: one-shot
|-----> filtering cells...
|-----> 2126 cells passed basic filters.
|-----> filtering gene...
|-----> 13855 genes passed basic filters.
|-----> calculating size factor...
|-----> selecting genes in layer: X, sort method: SVR...
|-----> size factor normalizing the data, followed by log1p transformation.
|-----> Set <adata.X> to normalized data
|-----> applying PCA ...
|-----> <insert> X_pca to obsm in AnnData Object.
|-----> cell cycle scoring...
|-----> computing cell phase...
|-----? 
Dynamo is not able to perform cell cycle staging for you automatically. 
Since dyn.pl.phase_diagram in dynamo by default colors cells by its cell-cycle stage, 
you need to set color argument accordingly if confronting errors related to this.
|-----> [recipe_monocle preprocess] completed [

/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/preprocessing/utils.py:730: RuntimeWarning: invalid value encountered in divide
  var_ntr = adata.layers["new"].sum(0) / adata.layers["total"].sum(0)


|-----> [moments calculation] completed [22.2921s]
|-----> retrieve data for non-linear dimension reduction...
|-----> [UMAP] using X_pca with n_pca_components = 30
|-----> <insert> X_umap to obsm in AnnData Object.
|-----> [UMAP] completed [29.1992s]
|-----> accessing adj_matrix_key=connectivities built from args for clustering...
|-----> Start computing neighbor graph...
|-----------> X_data is None, fetching or recomputing...
|-----> fetching X data from layer:None, basis:pca
|-----> method arg is None, choosing methods automatically...
|-----------> method ball_tree selected
|-----> Detecting communities on graph...
|-----------> Converting graph_sparse_matrix to igraph object
|-----> [Community clustering with leiden] completed [13468.1433s]
|-----> [hdbscan density-based-clustering] completed [0.6697s]
|-----> accessing adj_matrix_key=connectivities built from args for clustering...
|-----> Detecting communities on graph...
|-----------> Converting graph_sparse_matrix to igraph o

calculating gene velocity vectors confidence based on phase portrait location with priors of progenitor/mature cell types: 2000it [00:40, 48.78it/s]
/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/tools/metric_velocity.py:436: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg = confidence.groupby("gene")["prog_confidence", "mature_confidence"].mean()
/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/tools/utils.py:2307: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var[store_key] = (


|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [12.2225s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.9597s]
|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [11.7569s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.9356s]
|-----> VectorField reconstruction begins...
|-----> Retrieve X and V based on basis: UMAP. 
        Vector field will be learned in the UMAP space.
|-----> Generating high dimensional grids and convert into a row matrix.
|-----> Learning vector 

In [14]:
cmap = plt.colormaps["tab20"]
palette = {
    "ICM-1": cmap(1), 
    "ICM-2": cmap(3),
    "EPI": cmap(5),
    "PE": cmap(7),
    "TE-1": cmap(9),
    "TE-2": cmap(11),
    "muralTE": cmap(13),
    "polarTE": cmap(15)
}
color = "CellType"
pointsize = 0.2
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7.5, 3))
dyn.pl.streamline_plot(adata, color=color, color_key=palette, pointsize=pointsize, ax=ax1, basis='umap', show_legend=False, save_show_or_return='return')
dyn.pl.streamline_plot(adata, color=color, color_key=palette, pointsize=pointsize, ax=ax2, basis='umap_rnd', show_legend='right', save_show_or_return='return')
ax1.set_title("Isoform-based")
ax2.set_title("Isoform-based (randomized)")
plt.tight_layout()
plt.savefig("figures/streamline_plot_transcript_based.png", dpi=300)
plt.close()

|-----------> plotting with basis key=X_umap
|-----------> skip filtering CellType by stack threshold when stacking color because it is not a numeric type
|-----------> plotting with basis key=X_umap_rnd
|-----------> skip filtering CellType by stack threshold when stacking color because it is not a numeric type


# Prepare data (scanpy umap)

In [15]:
adata = ad.read_h5ad("../08_blastocyst/results/scanpy_processed_transcript_based.h5ad")
adata.layers["new"] = adata.layers["nascent"]
adata = adata[(adata.obs["s4U"] == 400) & (adata.obs["Time"] == 3) & (adata.obs["ActD"].isna()) & (adata.obs["TC.Ratio"] >= 0)]
adata.write("results/dynamo_data_transcript_based.scanpy_umap.h5ad", compression="gzip")
print(adata)

View of AnnData object with n_obs × n_vars = 2130 × 50565
    obs: 'Run', 'Barcode', 'Species', 's4U', 'Time', 'ActD', 'Group', 'CellLine', 'Stage', 'Platform', 'Label', 'UMI', 'Cells', 'LibStruct', 'Remark', 'Cell.Reads', 'Trimmed.Reads', 'Trimmed.Ratio', 'Mapped.Reads', 'Mapped.Ratio', 'Mito.Ratio', 'Filtered.Reads', 'Filtered.Ratio', 'FilteredClip.Reads', 'FilteredClip.Ratio', 'UMIs', 'UMIs.2Reads', 'Duplicate.Reads', 'Duplicate.Ratio', 'Unique.Reads', 'Genes', 'Isoforms.Assembled', 'Isoforms.Known', 'AC.Ratio', 'AG.Ratio', 'AT.Ratio', 'CA.Ratio', 'CG.Ratio', 'CT.Ratio', 'GA.Ratio', 'GC.Ratio', 'GT.Ratio', 'TA.Ratio', 'TC.Ratio', 'TG.Ratio', 'Pe', 'Pc', 'PcPe.Ratio', 'UMIs.2Reads.Nascent.2TCs', 'UMIs.2Reads.Nascent.2TCs.Ratio', 'Genes.2Reads', 'Genes.2Reads.Nascent.2TCs', 'n_genes', 'n_counts', 'leiden', 'CellType'
    var: 'TranscriptID', 'TranscriptType', 'GeneID', 'GeneName', 'GeneType', 'Chrom', 'Start', 'End', 'Strand', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dis

# Run dynamo (scanpy umap)

In [16]:
adata = ad.read_h5ad("results/dynamo_data_transcript_based.scanpy_umap.h5ad")
dyn.pp.recipe_monocle(adata, tkey="Time")
dyn.tl.dynamics(adata, model='stochastic', cores=3, NTR_vel=True)
dyn.tl.reduceDimension(adata)
dyn.tl.leiden(adata)
dyn.tl.hdbscan(adata)
dyn.tl.louvain(adata)
if True:
    lineage_dict = {"TE-1": ["muralTE", "polarTE"], "ICM-1": ["EPI", "PE"]}
    dyn.tl.gene_wise_confidence(adata, group='CellType', lineage_dict=lineage_dict, ekey="M_t", vkey="velocity_T")
    adata = adata[:,(adata.var["avg_prog_confidence"] > 0) & (adata.var["avg_mature_confidence"] > 0.5)]
dyn.tl.cell_velocities(adata, calc_rnd_vel=True)
dyn.vf.VectorField(adata, basis='umap')

|-----? dynamo.preprocessing.deprecated is deprecated.
|-----> recipe_monocle_keep_filtered_cells_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_cells_key=True
|-----> recipe_monocle_keep_filtered_genes_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_genes_key=True
|-----> recipe_monocle_keep_raw_layers_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_raw_layers_key=True
|-----> apply Monocole recipe to adata...


/home/chenzonggui/tmp/ipykernel_17708/720758844.py:2: DeprecationWarning: recipe_monocle is deprecated and will be removed in a future release. Please update your code to use the new replacement function.
  dyn.pp.recipe_monocle(adata, tkey="Time")


|-----> ensure all cell and variable names unique.
|-----> ensure all data in different layers in csr sparse matrix format.
|-----> ensure all labeling data properly collapased
|-----> detected experiment type: one-shot
|-----> filtering cells...
|-----> 2130 cells passed basic filters.
|-----> filtering gene...
|-----> 15147 genes passed basic filters.
|-----> calculating size factor...
|-----> selecting genes in layer: X, sort method: SVR...
|-----> size factor normalizing the data, followed by log1p transformation.
|-----> Set <adata.X> to normalized data
|-----> applying PCA ...
|-----> <insert> X_pca to obsm in AnnData Object.
|-----> cell cycle scoring...
|-----> computing cell phase...
|-----? 
Dynamo is not able to perform cell cycle staging for you automatically. 
Since dyn.pl.phase_diagram in dynamo by default colors cells by its cell-cycle stage, 
you need to set color argument accordingly if confronting errors related to this.
|-----> [recipe_monocle preprocess] completed [

/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/preprocessing/utils.py:730: RuntimeWarning: invalid value encountered in divide
  var_ntr = adata.layers["new"].sum(0) / adata.layers["total"].sum(0)


|-----> [moments calculation] completed [22.5437s]
|-----> retrieve data for non-linear dimension reduction...
|-----? adata already have basis umap. dimension reduction umap will be skipped! 
set enforce=True to re-performing dimension reduction.
|-----> [UMAP] completed [0.0027s]
|-----> accessing adj_matrix_key=connectivities built from args for clustering...
|-----> Detecting communities on graph...
|-----------> Converting graph_sparse_matrix to igraph object
|-----> [Community clustering with leiden] completed [625.8660s]
|-----> [hdbscan density-based-clustering] completed [3.3419s]
|-----> accessing adj_matrix_key=connectivities built from args for clustering...
|-----> Detecting communities on graph...
|-----------> Converting graph_sparse_matrix to igraph object
|-----? louvain is not maintained, we recommend using leiden instead.
|-----> [Community clustering with louvain] completed [3.4961s]


calculating gene velocity vectors confidence based on phase portrait location with priors of progenitor/mature cell types: 2000it [00:38, 52.09it/s]
/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/tools/metric_velocity.py:436: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg = confidence.groupby("gene")["prog_confidence", "mature_confidence"].mean()
/home/chenzonggui/miniconda3/envs/py3/lib/python3.10/site-packages/dynamo/tools/utils.py:2307: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var[store_key] = (


|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [4.7960s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.9003s]
|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [4.7535s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [0.9388s]
|-----> VectorField reconstruction begins...
|-----> Retrieve X and V based on basis: UMAP. 
        Vector field will be learned in the UMAP space.
|-----> Generating high dimensional grids and convert into a row matrix.
|-----> Learning vector fi

In [17]:
cmap = plt.colormaps["tab20"]
palette = {
    "ICM-1": cmap(1), 
    "ICM-2": cmap(3),
    "EPI": cmap(5),
    "PE": cmap(7),
    "TE-1": cmap(9),
    "TE-2": cmap(11),
    "muralTE": cmap(13),
    "polarTE": cmap(15)
}
color = "CellType"
pointsize = 0.2
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7.5, 3))
dyn.pl.streamline_plot(adata, color=color, color_key=palette, pointsize=pointsize, ax=ax1, basis='umap', show_legend=False, save_show_or_return='return')
dyn.pl.streamline_plot(adata, color=color, color_key=palette, pointsize=pointsize, ax=ax2, basis='umap_rnd', show_legend='right', save_show_or_return='return')
ax1.set_title("Isoform-based")
ax2.set_title("Isoform-based (randomized)")
plt.tight_layout()
plt.savefig("figures/streamline_plot_transcript_based.scanpy_umap.png", dpi=300)
plt.close()

|-----------> plotting with basis key=X_umap
|-----------> skip filtering CellType by stack threshold when stacking color because it is not a numeric type
|-----------> plotting with basis key=X_umap_rnd
|-----------> skip filtering CellType by stack threshold when stacking color because it is not a numeric type
